<a href="https://colab.research.google.com/github/racoope70/exploratory_daytrading/blob/main/PPO_QuantConnect_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Run to get paramaters for QuantConnect (from this codes output)
#get live_signals.json → the output your backtest consumes.

In [1]:
!pip -q uninstall -y opencv-python opencv-python-headless opencv-contrib-python

In [2]:
# Uninstall stuff that conflicts with a PyTorch+SB3+Gymnasium workflow (safe if absent)
!pip -q uninstall -y \
  gym gymnasium shimmy stable-baselines3 dopamine-rl \
  tensorflow tensorflow-hub tf-keras tensorflow-text tensorflow-decision-forests \
  cudf-cu12 dask-cudf-cu12 dask-cuda rapids-dask-dependency \
  libcudf-cu12 libcuml-cu12 pylibcudf-cu12 pylibraft-cu12 \
  libcugraph-cu12 pylibcugraph-cu12 rmm-cu12 libcuvs-cu12 cuvs-cu12 \
  cupy-cuda12x opencv-python opencv-python-headless opencv-contrib-python || true

# PyTorch (CUDA 12.4 wheels that match Colab GPU VMs)
!pip -q install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio

# Core RL/ML stack pinned to avoid Colab’s resolver nags
# - pandas==2.2.2 and requests==2.32.4 match google-colab’s constraints
# - numpy==2.0.2 avoids the OpenCV / numba complaints and works fine with SB3
!pip -q install -U \
  "protobuf>=5.29.1,<6" \
  "gymnasium>=1.1,<1.3" \
  "stable-baselines3==2.7.0" \
  "numpy==2.0.2" "pandas==2.2.2" "requests==2.32.4" \
  yfinance pywavelets transformers python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 944.3/944.3 kB 21.2 MB/s eta 0:00:00


In [ ]:
# Shell cell
!pip -q install "opencv-python-headless==4.12.0.88"

In [3]:
import glob, shutil
for p in glob.glob('/usr/local/lib/python*/dist-packages/~*'):
    print("Removing", p)
    shutil.rmtree(p, ignore_errors=True)


In [10]:
# --- SB3-safe cv2 shim: run this BEFORE `import stable_baselines3 as sb3`
import sys, types
try:
    import cv2  # Colab may auto-load a minimal cv2
except Exception:
    cv2 = None

if cv2 is None:
    cv2 = types.ModuleType("cv2")
    cv2.ocl = types.SimpleNamespace(setUseOpenCL=lambda *a, **k: None)
    sys.modules["cv2"] = cv2
else:
    if not hasattr(cv2, "ocl"):
        cv2.ocl = types.SimpleNamespace(setUseOpenCL=lambda *a, **k: None)
    elif not hasattr(cv2.ocl, "setUseOpenCL"):
        cv2.ocl.setUseOpenCL = lambda *a, **k: None

# Now safe to import SB3 & friends
import torch, gymnasium, stable_baselines3 as sb3, transformers, pandas as pd, numpy as np
print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| GPU:", torch.cuda.is_available())
print("Gymnasium:", gymnasium.__version__)
print("SB3:", sb3.__version__)
print("Transformers:", transformers.__version__)
print("pandas:", pd.__version__, "| numpy:", np.__version__)


Torch: 2.8.0+cu126 | CUDA: 12.6 | GPU: False
Gymnasium: 1.2.0
SB3: 2.7.0
Transformers: 4.56.1
pandas: 2.2.2 | numpy: 2.0.2


In [11]:
shimmy_needed = False
try:
    import gym_anytrading  # legacy gym-based
    shimmy_needed = True
    print("Detected gym-anytrading → Shimmy wrapper recommended.")
except Exception:
    print("gym-anytrading not installed → Shimmy not needed.")

print("Shimmy needed? ", shimmy_needed)


gym-anytrading not installed → Shimmy not needed.
Shimmy needed?  False


In [12]:
!pip -q install gym-anytrading shimmy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 2.8 MB/s eta 0:00:00


In [13]:
import os, json, shutil
from google.colab import files

# Where you keep all PPO artifacts
RESULTS_ROOT = "/content/drive/MyDrive/Results_May_2025"
MASTER_DIR   = os.path.join(RESULTS_ROOT, "ppo_models_master")
os.makedirs(MASTER_DIR, exist_ok=True)

# Pick a symbol/prefix you’re uploading for (repeat for CVX, etc.)
PREFIX = "ppo_GE_window1"   # change to "ppo_CVX_window1" when uploading CVX files

# === 1) Upload PPO artifacts from your local machine ===
# Expecting files named exactly like:
#   ppo_GE_window1_model.zip
#   ppo_GE_window1_vecnorm.pkl
#   ppo_GE_window1_features.json
# (optionally) ppo_GE_window1_probability_config.json, ppo_GE_window1_model_info.json
uploaded = files.upload()   # choose the files above

# Persist uploads to your MASTER_DIR
for name, data in uploaded.items():
    # Ensure file exists on the runtime filesystem (Colab sometimes does this automatically)
    with open(name, "wb") as f:
        f.write(data)
    # Move into your master artifacts folder
    shutil.move(name, os.path.join(MASTER_DIR, name))

print("Saved to:", MASTER_DIR)
print(sorted([f for f in os.listdir(MASTER_DIR) if f.startswith(PREFIX)]))

Saving ppo_GE_window1_features.json to ppo_GE_window1_features.json
Saving ppo_GE_window1_model_info.json to ppo_GE_window1_model_info.json
Saving ppo_GE_window1_model.zip to ppo_GE_window1_model.zip
Saving ppo_GE_window1_probability_config.json to ppo_GE_window1_probability_config.json
Saving ppo_GE_window1_vecnorm.pkl to ppo_GE_window1_vecnorm.pkl
Saved to: /content/drive/MyDrive/Results_May_2025/ppo_models_master
['ppo_GE_window1_features.json', 'ppo_GE_window1_model.zip', 'ppo_GE_window1_model_info.json', 'ppo_GE_window1_probability_config.json', 'ppo_GE_window1_vecnorm.pkl']


In [14]:
# 1) Upload your local env file (e.g., Github.env.txt or .env.github)
from google.colab import files
uploaded = files.upload()   # run this cell and choose the file

Saving Github_key.env.txt to Github_key.env.txt


In [15]:

# 2) Rename to .env (only if your filename isn't already ".env")
import os
if "Github_key.env.txt" in uploaded:
    os.rename("Github_key.env.txt", ".env")    # adjust if your uploaded name differs


In [16]:
import os, json, shutil

RESULTS_ROOT = "/content/drive/MyDrive/Results_May_2025"
MASTER_DIR   = os.path.join(RESULTS_ROOT, "ppo_models_master")
os.makedirs(MASTER_DIR, exist_ok=True)

PREFIX = "ppo_GE_window1"  # change to ppo_CVX_window1 for CVX

# Only upload if the key files are missing
needed = [f"{PREFIX}_model.zip", f"{PREFIX}_vecnorm.pkl", f"{PREFIX}_features.json"]
missing = [f for f in needed if not os.path.exists(os.path.join(MASTER_DIR, f))]
if missing:
    from google.colab import files
    print("Missing:", missing, "\nPlease upload the listed files.")
    uploaded = files.upload()
    for name, data in uploaded.items():
        with open(name, "wb") as f:
            f.write(data)
        shutil.move(name, os.path.join(MASTER_DIR, name))
    print("Saved to:", MASTER_DIR)

# Token: prefer .env OR set once in-session, not both
token = os.getenv("GITHUB_TOKEN")
if not token:
    try:
        # If you keep secrets in a .env file on Drive:
        # !pip -q install python-dotenv
        from dotenv import load_dotenv
        load_dotenv(".env")
        token = os.getenv("GITHUB_TOKEN")
    except Exception:
        pass

if not token:
    # Last-resort secure prompt (doesn't echo)
    from getpass import getpass
    token = getpass("Paste your GitHub token (won't be printed): ").strip()
    os.environ["GITHUB_TOKEN"] = token

print("Artifacts present:", all(os.path.exists(os.path.join(MASTER_DIR, f)) for f in needed))
print("Token present?   ", bool(os.getenv("GITHUB_TOKEN")))


Artifacts present: True
Token present?    True


In [17]:
# -------------------------------------- 3) IMPORTS -------------------------------------------
import os, json, time, logging
from datetime import datetime, timedelta, timezone
from typing import Dict, Any, List

import numpy as np
import pandas as pd
import requests
import yfinance as yf

import torch
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

from dotenv import load_dotenv

# gym-anytrading (legacy Gym API); harmless banner may print on import
from gym_anytrading.envs import StocksEnv
from gymnasium.spaces import Box as GBox

# ---------------------------------- 4) LOGGING & COLAB DRIVE ---------------------------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
L = logging.getLogger("producer")

try:
    from google.colab import drive  # type: ignore
    if not os.path.ismount("/content/drive"):
        drive.mount("/content/drive")
except Exception:
    pass

# ---------------------------------- 5) PATHS & CONFIG ----------------------------------------
RESULTS_ROOT = "/content/drive/MyDrive/Results_May_2025"
MASTER_DIR   = os.path.join(RESULTS_ROOT, "ppo_models_master")   # where <prefix>_model.zip etc. live
os.makedirs(MASTER_DIR, exist_ok=True)

STATE_DIR    = "/content/drive/MyDrive/QuantConnect_Ready"
os.makedirs(STATE_DIR, exist_ok=True)
GIST_ID_PATH = os.path.join(STATE_DIR, "live_signals_gist_id.txt")

# Which models to publish
PICKS: Dict[str, str] = {
    "GE":  "ppo_GE_window1",
    "CVX": "ppo_CVX_window1",    # leave in; will gracefully warn if artifacts missing
}

# yfinance fetch params
YF_INTERVAL = "1m"   # 1-minute bars
YF_DAYS     = 5      # yfinance supports about 7 days of 1m history

# ---------------------------------- 6) AUTH / SECRETS ----------------------------------------
# Load .env if present; otherwise you can set os.environ["GITHUB_TOKEN"] = "ghp_..."
if os.path.exists(".env"):
    load_dotenv(".env")

GITHUB_TOKEN  = os.environ.get("GITHUB_TOKEN", "").strip()
GIST_ID       = os.environ.get("GIST_ID", "").strip()        # optional override; else persisted to file
GIST_FILENAME = "live_signals.json"
GIST_DESC     = "Live PPO signals for QC (Producer→Consumer)"

# ---------------------------------- 7) OPTIONAL UPLOAD (INTERACTIVE) -------------------------
# If you need to upload artifacts from your local machine, set this True and follow the prompt.
ENABLE_INTERACTIVE_UPLOAD = False
UPLOAD_PREFIX = "ppo_GE_window1"   # change to "ppo_CVX_window1" to upload CVX

if ENABLE_INTERACTIVE_UPLOAD:
    try:
        from google.colab import files  # type: ignore
        needed = [f"{UPLOAD_PREFIX}_model.zip", f"{UPLOAD_PREFIX}_vecnorm.pkl", f"{UPLOAD_PREFIX}_features.json"]
        missing = [f for f in needed if not os.path.exists(os.path.join(MASTER_DIR, f))]
        if missing:
            print("Missing files:", missing)
            print("Please upload the listed files (they will be moved into ppo_models_master).")
            uploaded = files.upload()
            for name, data in uploaded.items():
                with open(name, "wb") as f:
                    f.write(data)
                shutil.move(name, os.path.join(MASTER_DIR, name))
            print("Saved to:", MASTER_DIR)
    except Exception as e:
        print("Upload step skipped or failed:", e)

# ---------------------------------- 8) FEATURE PIPELINE (DUMMY) ------------------------------
# Replace with your real feature computation if you have one.
def compute_enhanced_features(df_in: pd.DataFrame) -> pd.DataFrame:
    return df_in

# ---------------------------------- 9) TRADING ENV (StocksEnv → Gymnasium) -------------------
class ContinuousPositionEnv(StocksEnv):
    """
    Wraps gym_anytrading.envs.StocksEnv (legacy Gym) but returns Gymnasium-style (obs, reward, terminated, truncated, info).
    Action: continuous position target in [-1, +1].
    """
    def __init__(self, df, frame_bound, window_size,
                 cost_rate=0.0002, slip_rate=0.0003,
                 k_alpha=0.20, k_mom=0.05, k_sent=0.0,
                 mom_source="denoised", mom_lookback=20,
                 min_trade_delta=0.01, cooldown=5, reward_clip=1.0):
        super().__init__(df=df.reset_index(drop=True), frame_bound=frame_bound, window_size=window_size)
        # Ensure spaces are Gymnasium Boxes
        if isinstance(self.observation_space, gym.spaces.Box):
            self.observation_space = GBox(
                low=self.observation_space.low,
                high=self.observation_space.high,
                shape=self.observation_space.shape,
                dtype=self.observation_space.dtype,
            )
        self.action_space = GBox(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)

        self.cost_rate, self.slip_rate = float(cost_rate), float(slip_rate)
        self.k_alpha, self.k_mom = float(k_alpha), float(k_mom)
        self.k_sent = float(k_sent)
        self.mom_source, self.mom_lookback = str(mom_source), int(mom_lookback)
        self.min_trade_delta, self.cooldown = float(min_trade_delta), int(cooldown)
        self.reward_clip = float(reward_clip)

        self.nav, self.pos, self._last_trade_step = 1.0, 0.0, -self.cooldown

    def reset(self, **kwargs):
        out = super().reset(**kwargs)
        obs, info = (out if isinstance(out, tuple) else (out, {}))
        self.nav, self.pos, self._last_trade_step = 1.0, 0.0, -self.cooldown
        info = info or {}
        info.update({"nav": self.nav, "pos": self.pos})
        return obs, info

    def _step_parent_hold(self):
        # Parent env uses discrete actions; action 2 = HOLD (no-op)
        step_result = super().step(2)
        if len(step_result) == 5:  # already Gymnasium format
            obs, _env_rew, terminated, truncated, info = step_result
        else:                      # legacy Gym 4-tuple
            obs, _env_rew, done, info = step_result
            terminated, truncated = bool(done), False
        return obs, terminated, truncated, info

    def _ret_t(self):
        cur  = float(self.df.loc[self._current_tick, 'Close'])
        prev = float(self.df.loc[max(self._current_tick - 1, 0), 'Close'])
        return 0.0 if prev <= 0 else (cur - prev) / prev

    def _mom_signal(self):
        if self.mom_source == "macd" and "MACD_Line" in self.df.columns:
            recent = self.df["MACD_Line"].iloc[max(self._current_tick-200,0):self._current_tick+1]
            return float(np.tanh(float(self.df.loc[self._current_tick, "MACD_Line"]) / (1e-6 + recent.std())))
        if "Denoised_Close" in self.df.columns and self._current_tick - self.mom_lookback >= 0:
            now  = float(self.df.loc[self._current_tick, "Denoised_Close"])
            then = float(self.df.loc[self._current_tick - self.mom_lookback, "Denoised_Close"])
            base = float(self.df.loc[max(self._current_tick - 1, 0), "Close"])
            slope = (now - then) / max(self.mom_lookback, 1)
            return float(np.tanh(10.0 * (slope / max(abs(base), 1e-6))))
        return 0.0

    def step(self, action):
        a = float(np.array(action).squeeze())
        target_pos = float(np.clip(a, -1.0, 1.0))

        r_t = self._ret_t()
        base_ret = self.pos * r_t

        changed = (abs(target_pos - self.pos) >= self.min_trade_delta) and \
                  ((self._current_tick - self._last_trade_step) >= self.cooldown)
        delta_pos = (target_pos - self.pos) if changed else 0.0
        trade_cost = (self.cost_rate + self.slip_rate) * abs(delta_pos)

        rel_alpha  = base_ret - r_t
        mom_term   = self.pos * self._mom_signal()
        shaped     = base_ret + self.k_alpha*rel_alpha + self.k_mom*mom_term - trade_cost
        reward     = float(np.clip(shaped, -self.reward_clip, self.reward_clip))

        self.nav  *= (1.0 + base_ret - trade_cost)
        if changed:
            self.pos = target_pos
            self._last_trade_step = self._current_tick

        obs, terminated, truncated, info = self._step_parent_hold()
        info = info or {}
        info.update({"ret_t": r_t, "nav": self.nav, "pos": self.pos,
                     "trade_cost": trade_cost, "base_ret": base_ret,
                     "rel_alpha": rel_alpha, "mom": self._mom_signal()})
        return obs, reward, terminated, truncated, info

# ------------------------------- 10) ARTIFACT HELPERS ----------------------------------------
def _features_list_for(prefix: str) -> List[str]:
    fpath = os.path.join(MASTER_DIR, f"{prefix}_features.json")
    if os.path.exists(fpath):
        try:
            meta = json.load(open(fpath, "r"))
            feats = meta.get("features") or []
            if isinstance(feats, list):
                return [str(c) for c in feats]
        except Exception as e:
            L.warning(f"features.json read failed for {prefix}: {e}")
    return []

def _align_columns(df: pd.DataFrame, prefix: str) -> pd.DataFrame:
    feats = _features_list_for(prefix)
    if not feats:
        return df
    aligned = df.copy()
    for c in feats:
        if c not in aligned.columns:
            aligned[c] = 0.0
    ordered = [c for c in feats if c in aligned.columns] + [c for c in aligned.columns if c not in feats]
    return aligned[ordered]

def _check_artifacts(prefix: str) -> Dict[str, bool]:
    need = ["_model.zip", "_vecnorm.pkl"]
    nice = ["_features.json", "_probability_config.json", "_model_info.json"]
    return {s: os.path.exists(os.path.join(MASTER_DIR, prefix + s)) for s in need + nice}

# -------------------------------- 11) MODEL / ENV LOADING ------------------------------------
def get_mu_sigma(model, obs):
    with torch.no_grad():
        obs_t, _     = model.policy.obs_to_tensor(obs)
        feats        = model.policy.extract_features(obs_t)
        latent_pi, _ = model.policy.mlp_extractor(feats)
        mean_actions = model.policy.action_net(latent_pi)
        log_std      = model.policy.log_std
        mu    = float(mean_actions.detach().cpu().numpy().squeeze())
        sigma = float(log_std.exp().detach().cpu().numpy().squeeze())
    return mu, sigma

def load_model_and_env(prefix: str):
    model_path = os.path.join(MASTER_DIR, f"{prefix}_model.zip")
    vec_path   = os.path.join(MASTER_DIR, f"{prefix}_vecnorm.pkl")
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Missing model: {model_path}")
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = PPO.load(model_path, device=device)

    def make_env(df_window: pd.DataFrame):
        # simple frame bound near the end
        frame_bound = (max(50, len(df_window)//3), len(df_window) - 3)
        e = DummyVecEnv([lambda: ContinuousPositionEnv(
            df=df_window, frame_bound=frame_bound, window_size=10,
            cost_rate=0.0002, slip_rate=0.0003,
            k_alpha=0.20, k_mom=0.05, k_sent=0.0,
            mom_source="denoised", mom_lookback=20,
            min_trade_delta=0.01, cooldown=5, reward_clip=1.0
        )])
        if os.path.exists(vec_path):
            e = VecNormalize.load(vec_path, e)
        e.training = False
        e.norm_reward = False
        return e
    return model, make_env

# -------------------------------- 12) DATA FETCH / PREP --------------------------------------
def _flatten_yf_columns(df: pd.DataFrame, symbol: str) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        new_cols = []
        for c0, c1 in df.columns.to_list():
            if c1 in ("", symbol):
                new_cols.append(c0)
            else:
                new_cols.append(f"{c0}_{c1}")
        df.columns = new_cols
    return df

def latest_df_for_symbol(symbol: str, horizon_days: int = YF_DAYS, interval: str = YF_INTERVAL) -> pd.DataFrame | None:
    end   = datetime.now(timezone.utc)
    start = end - timedelta(days=horizon_days)
    df = yf.download(symbol,
                     start=start.strftime("%Y-%m-%d"),
                     end=end.strftime("%Y-%m-%d"),
                     interval=interval,
                     progress=False,
                     auto_adjust=False)
    if df is None or df.empty:
        return None
    df = df.reset_index()
    df["Symbol"] = symbol
    df = _flatten_yf_columns(df, symbol)
    df = compute_enhanced_features(df)
    return df

# -------------------------------- 13) INFERENCE ----------------------------------------------
def predict_latest(symbol: str, prefix: str) -> Dict[str, Any]:
    status = _check_artifacts(prefix)
    missing_hard = [k for k in ["_model.zip", "_vecnorm.pkl"] if not status.get(k, False)]
    if missing_hard:
        return {"symbol": symbol, "prefix": prefix, "error": f"missing artifacts: {missing_hard}"}

    model, make_env = load_model_and_env(prefix)
    live_df = latest_df_for_symbol(symbol)
    if live_df is None or len(live_df) < 120:
        return {"symbol": symbol, "prefix": prefix, "error": "no fresh data"}

    live_df = _align_columns(live_df, prefix)
    df_window = live_df.iloc[-2500:].reset_index(drop=True) if len(live_df) > 2500 else live_df.copy()

    env = make_env(df_window)
    obs = env.reset()
    if isinstance(obs, tuple):
        obs, _ = obs

    # Roll forward with a neutral policy to get to the end-of-window observation
    for _ in range(max(1, len(df_window) - 5)):
        obs, _, dones, _ = env.step([np.array([0.0], dtype=np.float32)])  # (n_envs, action_dim)
        if isinstance(dones, (np.ndarray, list)) and bool(dones[0]):
            break

    action, _ = model.predict(obs, deterministic=True)
    mu, sigma = get_mu_sigma(model, obs)
    from math import erf, sqrt
    Phi   = lambda x: 0.5 * (1.0 + erf(x / sqrt(2.0)))
    p_long = 1.0 - Phi((0.0 - mu) / max(sigma, 1e-6))

    a = float(np.array(action).squeeze())
    signal = "BUY" if a > 0.10 else ("SELL" if a < -0.30 else "HOLD")
    ts = df_window["Datetime"].iloc[-1] if "Datetime" in df_window.columns else None
    px = float(df_window["Close"].iloc[-1])

    return {
        "symbol": symbol,
        "prefix": prefix,
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "bar_ts": str(ts),
        "price": px,
        "action": a,
        "signal": signal,
        "confidence": abs(a),
        "p_long": float(p_long),
        "p_short": float(1.0 - p_long),
        "mu": float(mu),
        "sigma": float(sigma)
    }

# -------------------------------- 14) GIST HELPERS (ROBUST) ----------------------------------
def _headers(token: str) -> Dict[str, str]:
    return {"Authorization": f"token {token}"} if token else {}

def _load_saved_gist_id() -> str:
    if GIST_ID:
        return GIST_ID
    try:
        gid = open(GIST_ID_PATH, "r").read().strip()
        return gid if gid else ""
    except Exception:
        return ""

def _save_gist_id(gid: str):
    try:
        with open(GIST_ID_PATH, "w") as f:
            f.write(gid)
    except Exception:
        pass

def publish_json_to_gist(payload: dict, filename: str, gist_id: str, token: str, desc: str) -> dict:
    """
    Returns: {"id": <gist_id>, "owner": <login or 'anonymous'>, "raw_url": <raw file url or ''>}
    """
    if not token:
        raise RuntimeError("GITHUB_TOKEN not set. Set os.environ['GITHUB_TOKEN']='ghp_...' or create a .env first.")

    files = {filename: {"content": json.dumps(payload, indent=2)}}

    if gist_id:
        r = requests.patch(f"https://api.github.com/gists/{gist_id}",
                           headers=_headers(token),
                           json={"files": files, "description": desc}, timeout=30)
    else:
        r = requests.post("https://api.github.com/gists",
                          headers=_headers(token),
                          json={"files": files, "description": desc, "public": True},
                          timeout=30)

    if not r.ok:
        raise RuntimeError(f"Gist API error {r.status_code}: {r.text[:300]}")

    data = r.json()
    gid = data.get("id", gist_id or "")
    owner = ((data.get("owner") or {}).get("login")) or "anonymous"
    raw_url = ((data.get("files") or {}).get(filename) or {}).get("raw_url", "")

    if not gid:
        raise RuntimeError("Gist ID missing in API response.")

    if not raw_url:
        m = requests.get(f"https://api.github.com/gists/{gid}", headers=_headers(token), timeout=30)
        if m.ok and m.headers.get("content-type","").startswith("application/json"):
            md = m.json()
            raw_url = ((md.get("files") or {}).get(filename) or {}).get("raw_url", "")

    _save_gist_id(gid)
    return {"id": gid, "owner": owner, "raw_url": raw_url}

def gist_raw_url(gist_id: str, filename: str, token: str = "") -> str:
    if not gist_id:
        return ""
    r = requests.get(f"https://api.github.com/gists/{gist_id}", headers=_headers(token), timeout=20)
    if not r.ok or not r.headers.get("content-type","").startswith("application/json"):
        return ""
    data = r.json()
    return ((data.get("files") or {}).get(filename) or {}).get("raw_url", "") or ""

# -------------------------------- 15) PUBLISH LOOP -------------------------------------------
RUN_LOOP  = False
SLEEP_SEC = 60

def run_once():
    # Optionally filter out symbols missing core artifacts so the Gist shows only valid models
    results = []
    for sym, pref in PICKS.items():
        try:
            out = predict_latest(sym, pref)
            if out.get("error"):
                L.warning(f"{sym} -> {out['error']}")
            results.append(out)
        except Exception as e:
            L.exception(f"{sym} predict error: {e}")
            results.append({"symbol": sym, "prefix": pref, "error": str(e)})

    payload = {
        "generated_utc": datetime.now(timezone.utc).isoformat(),
        "valid_until_utc": (datetime.now(timezone.utc) + timedelta(minutes=3)).isoformat(),
        "producer": "colab-sb3",
        "interval": YF_INTERVAL,
        "models": results
    }

    meta = publish_json_to_gist(payload, filename=GIST_FILENAME,
                                gist_id=_load_saved_gist_id(),
                                token=GITHUB_TOKEN, desc=GIST_DESC)

    gid = meta["id"]
    raw = meta.get("raw_url") or gist_raw_url(gid, GIST_FILENAME, token=GITHUB_TOKEN)

    print("Published:", f"https://gist.github.com/{gid}")
    print("RAW URL  :", raw or "(raw url not available yet — open gist page)")
    print("Preview  :", json.dumps(payload, indent=2)[:900], "...")

# -------------------------------- 16) SANITY CHECK + EXECUTE ---------------------------------
for sym, pref in PICKS.items():
    status = _check_artifacts(pref)
    hard_miss = [k for k in ["_model.zip", "_vecnorm.pkl"] if not status.get(k, False)]
    print(f"{pref}: {'OK' if not hard_miss else 'MISSING ' + str(hard_miss)}")

# Fallback prompt if token missing (won't echo); comment out if you prefer exception instead
if not GITHUB_TOKEN:
    try:
        from getpass import getpass
        os.environ["GITHUB_TOKEN"] = getpass("Paste your GitHub token (won't be printed): ").strip()
        GITHUB_TOKEN = os.environ["GITHUB_TOKEN"]
    except Exception:
        pass

run_once()

if RUN_LOOP:
    while True:
        try:
            run_once()
        except Exception as e:
            L.error(f"Publish error: {e}")
        time.sleep(SLEEP_SEC)

# -------------------------------- 17) OPTIONAL: PRINT STABLE RAW URL LATER -------------------
try:
    gid = open(GIST_ID_PATH, "r").read().strip()
    if gid:
        r = requests.get(f"https://api.github.com/gists/{gid}", headers=_headers(GITHUB_TOKEN), timeout=20)
        if r.ok and r.headers.get("content-type","").startswith("application/json"):
            meta = r.json()
            owner = (meta.get("owner") or {}).get("login", "anonymous")
            raw_url_stable = f"https://gist.githubusercontent.com/{owner}/{gid}/raw/{GIST_FILENAME}"
            raw_url_api    = (meta.get("files", {}).get(GIST_FILENAME, {}) or {}).get("raw_url", "")
            print("Gist page   :", f"https://gist.github.com/{owner}/{gid}")
            print("Raw (stable):", raw_url_stable)
            print("Raw (API)   :", raw_url_api)
        else:
            print("Gist meta fetch skipped (non-JSON or HTTP error).")
    else:
        print("No saved Gist ID yet.")
except Exception as e:
    print("Gist meta check skipped:", e)

ppo_GE_window1: OK
ppo_CVX_window1: MISSING ['_model.zip', '_vecnorm.pkl']


Published: https://gist.github.com/3deb62ce13c3eaa57b5d58c6eb858ce6
RAW URL  : https://gist.githubusercontent.com/racoope70/3deb62ce13c3eaa57b5d58c6eb858ce6/raw/75b5166bfb1cfadbf650c0ba594a70881e978ff4/live_signals.json
Preview  : {
  "generated_utc": "2025-09-13T20:47:27.699041+00:00",
  "valid_until_utc": "2025-09-13T20:50:27.699055+00:00",
  "producer": "colab-sb3",
  "interval": "1m",
  "models": [
    {
      "symbol": "GE",
      "prefix": "ppo_GE_window1",
      "timestamp": "2025-09-13T20:47:27.697474+00:00",
      "bar_ts": "2025-09-12 19:59:00+00:00",
      "price": 281.70001220703125,
      "action": 0.1543455868959427,
      "signal": "BUY",
      "confidence": 0.1543455868959427,
      "p_long": 0.5598957990575731,
      "p_short": 0.44010420094242686,
      "mu": 0.1543455868959427,
      "sigma": 1.0241568088531494
    },
    {
      "symbol": "CVX",
      "prefix": "ppo_CVX_window1",
      "error": "missing artifacts: ['_model.zip', '_vecnorm.pkl']"
    }
  ]
} ...
Gist